In [ ]:
import pandas as pd
import functions
df = pd.read_excel('concordance_preloaded_notenten23_tt1_20250617165050.xlsx')

In [2]:
df

,Reference,Sentence
0,"ht.no,2021-01-12 04:34",<s> Harstad-jenta har gått på Heggen videregåe...
1,"tu.no,2020-12-28 22:10","<s> Det vil si, de kan også gå på diesel . </s>"
2,"tu.no,2020-12-30 09:20",<s> De resterende nesten tusen bussene går på ...
3,"vi.no,2020-12-24 20:51",<s> I artikkelen forteller Elsa hvordan hun gj...
4,"ks.no,2020-12-22 12:03",<s> Når vi nå åpner opp for de resterende komm...
...,...,...
9995,"blogspot.com,2020-12-27 09:41",<s> Ingrid lagde Chili con carne med utgangspu...
9996,"blogspot.com,2020-12-24 11:09","<s> Noen går på terrassen her, noen på terrass..."
9997,"norwegianforum.net,2020-12-23 11:02","<s> Hun har nå gått på antibiotika i 14 dager,..."
9998,"norwegianforum.net,2020-12-24 12:28",<s> Når mine går på tredemølle bruker jeg førs...


In [3]:
df[['URL', 'Date']] = df['Reference'].str.split(',',expand=True).copy()

In [4]:
df = df.drop(columns=['Reference'])

In [ ]:
df['text'] = df['Sentence'].apply(functions.normalize_text)

In [8]:
import spacy
nlp = spacy.load("nb_core_news_sm")

In [27]:
def get_nouns_after_ga_pa(text):
    doc = nlp(text)
    for i in range(len(doc)-2):
        if doc[i].lemma_ == 'gå' and doc[i+1].text.lower() == 'på':
            j = i + 2
            if j < len(doc) and (doc[j].is_punct or doc[j].text.lower() in ['og', 'eller']):
                continue
            if j < len(doc) and doc[j].pos_ == 'NOUN':
                return doc[j].lemma_ #return lemmatised noun
    return ''

df['nouns'] = df['text'].apply(get_nouns_after_ga_pa)

In [28]:
#nan values the df
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.head()

,Sentence,URL,Date,text,nouns
0,<s> Harstad-jenta har gått på Heggen videregåe...,ht.no,2021-01-12 04:34,Harstad-jenta har gått på Heggen videregående ...,NaN
1,"<s> Det vil si, de kan også gå på diesel . </s>",tu.no,2020-12-28 22:10,"Det vil si, de kan også gå på diesel .",diesel
2,<s> De resterende nesten tusen bussene går på ...,tu.no,2020-12-30 09:20,De resterende nesten tusen bussene går på dies...,diesel
3,<s> I artikkelen forteller Elsa hvordan hun gj...,vi.no,2020-12-24 20:51,I artikkelen forteller Elsa hvordan hun gjenno...,jobb
4,<s> Når vi nå åpner opp for de resterende komm...,ks.no,2020-12-22 12:03,Når vi nå åpner opp for de resterende kommunen...,papir


In [29]:
df.count()

Sentence    10000
URL         10000
Date        10000
text        10000
nouns        7428
dtype: int64

In [30]:
df['nouns'].value_counts()

nouns
skole              670
bekostning         536
ski                290
jobb               228
tvers              195
                  ... 
vareinformasjon      1
høn                  1
lovskol              1
frels                1
terrasse             1
Name: count, Length: 1784, dtype: int64

In [31]:
df.to_excel('gå_på_nouns.xlsx', index=False)